In [1]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m

@si.coroutine(inputs={"I": si.Bit})
def TFF(init=False):
    value = init
    O = value
    while True:
        I = yield O
        O = value
        value = I ^ O
        
TFF = si.compile(TFF(), file_name="build/silica_tff.py")
        
with open("build/silica_tff.py", "r") as magma_file:
    print(magma_file.read())

import mantle lattice ice40
import mantle lattice mantle40
from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator

TFF = DefineCircuit("TFF", "O", Out(Bit), "I", In(Bit), *ClockInterface(has_ce=False))
CE = VCC
I = TFF.I
value = DFF(has_ce=True, name="value", init=0)
value_CE = [CE]
wireclock(TFF, value)
value_next_0_tmp = value.O
O_0_tmp = value.O
value_next_0_tmp = xor(I, O_0_tmp)
wire(value_next_0_tmp, value.I)
wire(O_0_tmp, TFF.O)
if len(value_CE) == 1:
    wire(value_CE[0], value.CE)
else:
    wire(and_(*value_CE), value.CE)
EndDefine()


In [2]:
!magma -o verilog -m lattice -t TFF build/silica_tff.py
!yosys -p 'synth_ice40 -top TFF -blif build/silica_tff.blif' build/silica_tff.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -q -d 1k -o build/silica_tff.txt build/silica_tff.blif
!icetime -tmd hx1k build/silica_tff.txt | grep -B 2 "Total path delay"

import mantle lattice ice40
import mantle lattice mantle40
compiling XOr2
compiling TFF
2.27. Printing statistics.

=== TFF ===

   Number of wires:                  8
   Number of wire bits:              9
   Number of public wires:           8
   Number of public wire bits:       9
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  2
     SB_DFFE                         1
     SB_LUT4                         1


Total number of logic levels: 1
Total path delay: 1.61 ns (621.93 MHz)


In [3]:
from mantle import DFF

TFF = m.DefineCircuit('TFF', 'I', m.In(m.Bit), 'O', m.Out(m.Bit), 'CLK', m.In(m.Bit))

# create ff holding state first
ff = DFF()

# compute the next state
ff( TFF.I ^ ff.O )

m.wire(ff.O, TFF.O)
m.wire(TFF.CLK, ff.CLK)

m.EndDefine()

m.compile("build/magma_tff", TFF)

compiling XOr2
compiling TFF


In [4]:
!yosys -p 'synth_ice40 -top TFF -blif build/magma_tff.blif' build/magma_tff.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -q -d 1k -o build/magma_tff.txt build/magma_tff.blif
!icetime -tmd hx1k build/magma_tff.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== TFF ===

   Number of wires:                  8
   Number of wire bits:              9
   Number of public wires:           8
   Number of public wire bits:       9
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  2
     SB_DFF                          1
     SB_LUT4                         1


Total number of logic levels: 1
Total path delay: 1.61 ns (621.93 MHz)
